In [32]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [33]:
def get_jobs(keyword, location, num_jobs, verbose, path, slp_time):
#def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)


    driver.get('https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&sc.keyword=&locT=&locId=&jobType=')
    
    #Uses first variable in function to input job title
    search = driver.find_element_by_id("KeywordSearch")
    search.send_keys(keyword)
    
    #Uses second variable in function to input Location.  Use 'City,State Abbreviation'
    search = driver.find_element_by_id("LocationSearch").clear()
    search = driver.find_element_by_id("LocationSearch")
    search.send_keys(location)
    search.send_keys(Keys.RETURN)
    
    #Test for the "Sign Up" prompt and get rid of it.
    time.sleep(slp_time)
    
    try:
                driver.find_element_by_class_name("selected").click()
    except ElementClickInterceptedException:
        pass
    
    time.sleep(.1)
    
    try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass
    
    #changes 'posted' dropdown to 'Last Week'
    driver.find_element_by_id('filter_fromAge').click()
    time.sleep(5)
    driver.find_element_by_xpath('.//ul[@class="css-1dv4b0s ew8xong0"]//li[@value="7"]').click()
   
    
    
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
       
        time.sleep(5)
        
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//div[@class = "salary"]').text

            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
               

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [35]:
path = "C:/Users/dell/Documents/ds_salary_proj/chromedriver"

df = get_jobs('data scientist', 'United States', 50, False, path, 15)

Progress: 0/50
Progress: 1/50
Progress: 2/50
Progress: 3/50
Progress: 4/50
Progress: 5/50
Progress: 6/50
Progress: 7/50
Progress: 8/50
Progress: 9/50
Progress: 10/50
Progress: 11/50
Progress: 12/50
Progress: 13/50
Progress: 14/50
Progress: 15/50
Progress: 16/50
Progress: 17/50
Progress: 18/50
Progress: 19/50
Progress: 20/50
Progress: 21/50
Progress: 22/50
Progress: 23/50
Progress: 24/50
Progress: 25/50
Progress: 26/50
Progress: 27/50
Progress: 28/50
Progress: 29/50
Progress: 30/50
Progress: 31/50
Progress: 32/50
Progress: 33/50
Progress: 34/50
Progress: 35/50
Progress: 36/50
Progress: 37/50
Progress: 38/50
Progress: 39/50
Progress: 40/50
Progress: 41/50
Progress: 42/50
Progress: 43/50
Progress: 44/50
Progress: 45/50
Progress: 46/50
Progress: 47/50
Progress: 48/50
Progress: 49/50
Progress: 50/50


In [36]:
df.to_csv('glassdoor_jobs.csv', index = False)

In [37]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Principal Data Scientist,$104K - $170K (Glassdoor est.),"Working out of our Chevy Chase, MD/Washington ...",3.5,GEICO\n3.5,"Chevy Chase, MD",10000+ Employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD)
1,"Scientist, Clinical Data Manager",$30K - $55K (Glassdoor est.),Job Number: 29829\n\nReynolds American Incorpo...,4.3,British American Tobacco\n4.3,"Winston-Salem, NC",10000+ Employees,1902,Company - Public,Consumer Products Manufacturing,Manufacturing,$10+ billion (USD)
2,Scientist,$51K - $108K (Glassdoor est.),Classification:\nSalary\nJob Description:\nAre...,3.5,Laticrete International\n3.5,"Grand Prairie, TX",201 to 500 Employees,-1,Company - Private,Chemical Manufacturing,Manufacturing,$100 to $500 million (USD)
3,Scientist - R&D ASC,$40K - $65K (Glassdoor est.),Job Number: 31115\n\nReynolds American Incorpo...,4.3,British American Tobacco\n4.3,"Memphis, TN",10000+ Employees,1902,Company - Public,Consumer Products Manufacturing,Manufacturing,$10+ billion (USD)
4,Operations Data Analyst,$36K - $62K (Glassdoor est.),Make your mark. With more than $17.2 billion o...,4.0,Plante Moran\n4.0,"Kalamazoo, MI",1001 to 5000 Employees,1924,Company - Private,Accounting,Accounting & Legal,$500 million to $1 billion (USD)
5,"Scientist III, Quality Control",$35K - $64K (Glassdoor est.),The Scientist 3 - QC is responsible to perform...,3.4,Amneal Pharmaceuticals\n3.4,"Brookhaven, NY",5001 to 10000 Employees,2002,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $2 billion (USD)
6,Risk Analytics Manager,-1,Key Responsibilities\nDevelop and execute risk...,-1,"Cane Bay Partners VI, LLP/CBP","Christiansted, VI",-1,-1,-1,-1,-1,-1
7,"AVP, Data Science for Personal Lines",$88K - $144K (Glassdoor est.),"AVP, Data Science for Personal Lines\nWorceste...",4.1,The Hanover Insurance Group\n4.1,"Worcester, MA",5001 to 10000 Employees,1852,Company - Public,Insurance Carriers,Insurance,$5 to $10 billion (USD)
8,"Sr Scientist, Product Data Management",$53K - $108K (Glassdoor est.),Job Number: 31636\n\nReynolds American Incorpo...,4.3,British American Tobacco\n4.3,"Winston-Salem, NC",10000+ Employees,1902,Company - Public,Consumer Products Manufacturing,Manufacturing,$10+ billion (USD)
9,Scientist,$41K - $91K (Glassdoor est.),Company Description\n\nEAG Laboratories is a g...,3.6,EAG Laboratories\n3.6,"Saint Louis, MO",501 to 1000 Employees,1978,Subsidiary or Business Segment,Research & Development,Business Services,$100 to $500 million (USD)
